In [1]:
import tensorflow as tf
import pandas as pd
import pickle
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
"""
Global values
"""
FEATURE_LEN = 5
folder = 'year2010_movie'


In [3]:

user_rating = pickle.load(open('../data/'+folder+'/user_rating.pkl', 'rb'))

user_rating_valid = pickle.load(open('../data/'+folder+'/user_rating_validation.pkl', 'rb'))

user_rating_test = pickle.load(open('../data/'+folder+'/user_rating_test.pkl', 'rb'))

df_train = []
max_user = 0
max_movie = 0
for user in user_rating:
    user_i = int(user)
    for tup in user_rating[user]:
        rating_i = float(tup[1])
        movie_i = int(tup[0])
        #user_list.append(user_i)
        #movie_list.append(movie_i)
        #rating_list.append(float(rating_i))
        df_train.append([user_i, movie_i, float(rating_i)])
        if movie_i > max_movie:
            max_movie = movie_i
    
    if user_i > max_user:
        max_user = user_i
        
df_test = []
for user in user_rating_test:
    user_i = int(user)
    for tup in user_rating_test[user]:
        rating_i = float(tup[1])
        movie_i = int(tup[0])
        #user_list.append(user_i)
        #movie_list.append(movie_i)
        #rating_list.append(float(rating_i))
        df_test.append([user_i, movie_i, float(rating_i)])
        if movie_i > max_movie:
            max_movie = movie_i
    
    if user_i > max_user:
        max_user = user_i
        

df_valid = []
for user in user_rating_valid:
    user_i = int(user)
    for tup in user_rating_valid[user]:
        rating_i = float(tup[1])
        movie_i = int(tup[0])
        #user_list.append(user_i)
        #movie_list.append(movie_i)
        #rating_list.append(float(rating_i))
        df_valid.append([user_i, movie_i, float(rating_i)])
        if movie_i > max_movie:
            max_movie = movie_i
    
    if user_i > max_user:
        max_user = user_i

#user_s, movie_s, rating_s = shuffle(user_list, movie_list, rating_list)

#df_train, df_test = train_test_split(df, test_size = 0.2)

"""
"""
user_s = [t[0] for t in df_train]
movie_s = [t[1] for t in df_train]
rating_s = [t[2] for t in df_train]

user_s_test = [t[0] for t in df_test]
movie_s_test = [t[1] for t in df_test]
rating_s_test = [t[2] for t in df_test]

user_s_valid = [t[0] for t in df_valid]
movie_s_valid = [t[1] for t in df_valid]
rating_s_valid = [t[2] for t in df_valid]

print("training number: ", len(user_s))
print("validaton number: ", len(user_s_valid))
print("test number: ", len(user_s_test))

training number:  201623
validaton number:  25203
test number:  25203


In [4]:
user_indecies_train = user_s
item_indecies_train = movie_s
rates_train = rating_s

user_indecies_test = user_s_test
item_indecies_test = movie_s_test
rates_test = rating_s_test

user_indecies_valid = user_s_valid
item_indecies_valid = movie_s_valid
rates_valid = rating_s_valid


In [5]:
max_movie

3906

In [6]:
totle_len = len(user_s)
totle_len

201623

In [7]:
def get_bias(user, movie, rating):
    print('max user:', max_user)
    print('max movie:', max_movie)
    average = np.mean(rating)
    user_dict = {}
    movie_dict = {}
    for i in range(len(user)):
        user_dict[user[i]] = [rating[i]] if user[i] not in user_dict else user_dict[user[i]] + [rating[i]]
        movie_dict[movie[i]] = [rating[i]] if movie[i] not in movie_dict else movie_dict[movie[i]] + [rating[i]]
    
    user_bias = [0 for i in range(max_user + 1)]
    movie_bias = [0 for i in range(max_movie + 1)]
    print(user_dict[2253])
    print(len(user_bias), len(movie_bias))
    for key in user_dict:
        user_bias[int(key)] = np.mean(user_dict[key]) - average
    for key in movie_dict:
        movie_bias[int(key)] = np.mean(movie_dict[key]) - average
    
    return user_bias, movie_bias, average

user_bias, movie_bias, average_score = get_bias(user_s, movie_s, rating_s)


    

max user: 3838
max movie: 3906
[2.0, 4.0, 4.0, 3.0, 4.0, 4.0, 2.0, 4.0, 3.5, 4.0, 4.5, 4.0, 4.0, 3.0, 2.0, 4.0, 4.0, 4.5, 4.5]
3839 3907


In [8]:
user_bias_r = np.reshape(user_bias,[-1,1])
movie_bias_r = np.reshape(movie_bias,[1,-1])

In [9]:
U = tf.Variable(initial_value=tf.truncated_normal([max_user+1, FEATURE_LEN], stddev=0.2, mean=0), name='users')
P = tf.Variable(initial_value=tf.truncated_normal([FEATURE_LEN, max_movie+1], stddev=0.2, mean=0), name='movies')

In [10]:

U_plus_bias = tf.concat([U, tf.constant(user_bias_r, dtype=tf.float32, name="user_bias"), tf.ones((max_user+1,1), dtype=tf.float32, name="user_bias_ones")], 1)
P_plus_bias = tf.concat([P, tf.ones((1, max_movie+1), name="movie_bias_ones", dtype=tf.float32), tf.constant(movie_bias_r, dtype=tf.float32, name="movie_bias")], 0)


In [11]:
#result = tf.matmul(U_plus_bias, P_plus_bias)
result = tf.matmul(U, P)

In [12]:
result_flatten = tf.reshape(result, [-1])
R = tf.gather(result_flatten, user_s * tf.shape(result)[1] + movie_s, name='extracting_user_rate')

In [13]:
diff_op = tf.subtract(R, rating_s, name='trainig_diff')
diff_op_squared = tf.nn.l2_loss(diff_op, name="squared_difference")
base_cost = tf.reduce_sum(diff_op_squared, name="sum_squared_error")

In [14]:
lda = tf.constant(0.001, name='lambda')
norm_sums = tf.add(tf.reduce_sum(tf.nn.l2_loss(U, name='user_abs'), name='user_norm'), 
   tf.reduce_sum(tf.nn.l2_loss(P, name='item_abs'), name='item_norm'))
regularizer = tf.multiply(norm_sums, lda, 'regularizer')

In [15]:
cost = tf.add(base_cost, regularizer)
#cost = base_cost

In [16]:
lr = tf.constant(.0001, name='learning_rate')
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(lr, global_step, 10000, 0.98, staircase=True)
#learning_rate = 0.01
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
training_step = optimizer.minimize(cost, global_step=global_step)

In [17]:


# accuracy
R_test = tf.gather(result_flatten, user_indecies_train * tf.shape(result)[1] + item_indecies_train, name='extracting_user_rate_test')
#R_test = tf.cast(R_test, tf.float64)
rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(rates_train, R_test))))
#print(sess.run(rmse))


In [18]:
for i in user_indecies_train:
    if i < 0:
        print(i)
        break

In [19]:


# accuracy
test_R_test = tf.gather(result_flatten, user_indecies_test * tf.shape(result)[1] + item_indecies_test, name='extracting_user_rate_test')
#test_R_test = tf.cast(test_R_test, tf.float64)
test_rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(rates_test, test_R_test))))
#print(sess.run(rmse))



In [20]:
# accuracy
valid_R_test = tf.gather(result_flatten, user_indecies_valid * tf.shape(result)[1] + item_indecies_valid, name='extracting_user_rate_valid')
#test_R_test = tf.cast(test_R_test, tf.float64)
valid_rmse = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(rates_valid, valid_R_test))))
#print(sess.run(rmse))

In [21]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
minVR = 999
not_change_n = 0
for i in range(10001):
    _, c, r, tr, vr = sess.run([training_step, cost, rmse, test_rmse, valid_rmse])
    if i%20 == 0:
        print(i, c, r, vr, tr, not_change_n)
    
        if vr + 0.001 < minVR:
            minVR = vr
            not_change_n = 0
        else:
            not_change_n += 1

        if not_change_n > 10:
            break
            
    
    """
    if i%500 == 0:
        r, tr = sess.run([rmse, test_rmse])
        #r = sess.run([rmse])
        print("rmse:", i, r, tr)
    """
    
    

0 1.29672e+06 3.58647 3.59135 3.58144 0
20 1.29423e+06 3.58303 3.59015 3.58017 0
40 1.27394e+06 3.55483 3.56529 3.55496 0
60 1.04233e+06 3.21548 3.24127 3.22794 0
80 510904.0 2.2512 2.29645 2.28605 0
100 303418.0 1.73485 1.7819 1.77576 0
120 219909.0 1.47693 1.52651 1.51843 0
140 177776.0 1.32792 1.37943 1.36924 0
160 152880.0 1.23143 1.28441 1.27288 0
180 136431.0 1.1633 1.21748 1.20519 0
200 124683.0 1.11208 1.16729 1.15466 0
220 115819.0 1.07182 1.12794 1.11524 0
240 108857.0 1.0391 1.09606 1.08346 0
260 103218.0 1.01183 1.0696 1.05721 0
280 98541.5 0.988641 1.0472 1.03508 0
300 94586.7 0.968597 1.02796 1.01614 0
320 91188.9 0.95104 1.01121 0.999707 0
340 88230.5 0.935483 0.996492 0.985302 0
360 85625.4 0.921567 0.983434 0.972555 0
380 83309.0 0.909014 0.97176 0.961183 0
400 81231.8 0.89761 0.961253 0.950967 0
420 79355.4 0.88718 0.951741 0.941734 0
440 77649.3 0.877589 0.943085 0.933346 0
460 76088.8 0.868725 0.935172 0.925687 0
480 74654.2 0.860495 0.927907 0.918665 0
500 73329.2 

KeyboardInterrupt: 

In [22]:
train_mae = tf.reduce_mean(tf.abs(tf.subtract(rates_train, R_test)))
test_mae = tf.reduce_mean(tf.abs(tf.subtract(rates_test, test_R_test)))
valid_mae = tf.reduce_mean(tf.abs(tf.subtract(rates_valid, valid_R_test)))
mae_tr, mae_v, mae_t = sess.run([train_mae, valid_mae, test_mae])
print(mae_tr)
print(mae_v)
print(mae_t)

0.485895
0.574433
0.575579


In [23]:
movie_rating = pickle.load(open('../data/'+folder+'/movie_rating.pkl', 'rb'))
movie_rating_test = pickle.load(open('../data/'+folder+'/movie_rating_test.pkl', 'rb'))

In [42]:
_U, _P, _result = sess.run([U,P, result])

In [55]:
import numpy as np
_p = np.transpose(_P)

In [83]:
total_not_in = 0
vv_predict = []
vv_y = []
largest_num = 10
_RMSE_list = []
for largest_num in range(0,200, 3): 
    for m in range(0, max_movie+1):
        if (m not in movie_rating or len(movie_rating[m]) <=largest_num) and m in movie_rating_test:
            #print('not in', m)
            #print(movie_rating_test[m])
            for pair in movie_rating_test[m]:
                movie_n = m
                user_n = pair[0]
                vv_predict.append(np.dot(_U[user_n], _p[movie_n]))
                vv_y.append(pair[1])
            total_not_in += len(movie_rating_test[m])

    _RMSE = np.sqrt(np.mean(np.square(np.subtract(vv_predict, vv_y))))
    _RMSE_list.append((largest_num, _RMSE))
    

In [73]:
total_not_in

969

In [74]:
np.sqrt(np.mean(np.square(np.subtract(vv_predict, vv_y))))

1.5459987380872264

In [75]:
_P[0][0], _P[1][0], _P[2][0], _P[3][0]

(-1.4057437, 0.66361493, -0.14481126, 0.74128443)

In [76]:
_p[0]

array([-1.40574372,  0.66361493, -0.14481126,  0.74128443,  0.15007383], dtype=float32)

In [84]:
_RMSE_list

[(0, 3.1833473659774985),
 (3, 2.4239389208122688),
 (6, 2.0946503370564096),
 (9, 1.8934228955623369),
 (12, 1.7639267280099786),
 (15, 1.6733536169803629),
 (18, 1.6004050526300291),
 (21, 1.539135741842548),
 (24, 1.4908253541431598),
 (27, 1.4511506977841526),
 (30, 1.4182021183114883),
 (33, 1.388515433223342),
 (36, 1.3633878015630871),
 (39, 1.3412227621970545),
 (42, 1.3216081509387461),
 (45, 1.3035674169628577),
 (48, 1.2872281372444101),
 (51, 1.2729777264621029),
 (54, 1.2598103481206395),
 (57, 1.246946441716122),
 (60, 1.234420584129059),
 (63, 1.2229276118025165),
 (66, 1.2120673221649481),
 (69, 1.2022890595809512),
 (72, 1.1930715842642268),
 (75, 1.1845622956478419),
 (78, 1.1765066902508297),
 (81, 1.1690175938434828),
 (84, 1.1618830059701983),
 (87, 1.1550936105798155),
 (90, 1.148503215132646),
 (93, 1.1423363927610246),
 (96, 1.1362726292596412),
 (99, 1.1304295603454488),
 (102, 1.1248971903410017),
 (105, 1.1197569407728969),
 (108, 1.1148025739765144),
 (111, 

In [85]:
import csv
with open('MF_RMSE.csv','w') as f:
    f_csv = csv.writer(f)
    headers = ['num, RMSE']
    f_csv.writerow(headers)
    for p in _RMSE_list:
        row = [p[0], p[1]]
        f_csv.writerow(row)